# Fine tuning

In this notebook, we finetune an opensource sentencetransformers embedding model on our synthetically generated dataset.
En este notebook, ajustamos un modelo de integración de transformadores de oraciones de código abierto en nuestro conjunto de datos generado sintéticamente.

### Cargar modelo pre-entrenado

In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
model_id = "BAAI/bge-small-en"
model = SentenceTransformer(model_id)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [3]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

### Define dataloader

In [4]:
import json

from torch.utils.data import DataLoader
from sentence_transformers import InputExample

In [5]:
TRAIN_DATASET_FPATH = './data/train_dataset.json'
VAL_DATASET_FPATH = './data/val_dataset.json'

# Usamos un tamaño de lote muy pequeño para ejecutar este ejemplo de prueba en una máquina local.
# Normalmente debería ser mucho más grande.

BATCH_SIZE = 10

In [6]:
with open(TRAIN_DATASET_FPATH, 'r+') as f:
    train_dataset = json.load(f)

with open(VAL_DATASET_FPATH, 'r+') as f:
    val_dataset = json.load(f)

In [7]:
dataset = train_dataset

corpus = dataset['corpus']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

examples = []
for query_id, query in queries.items():
    node_id = relevant_docs[query_id][0]
    text = corpus[node_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [8]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

### Definir pérdida

**MultipleNegativesRankingLoss** is a great loss function if you only have positive pairs, for example, only pairs of similar texts like pairs of paraphrases, pairs of duplicate questions, pairs of (query, response), or pairs of (source_language, target_language).

This loss function works great to train embeddings for retrieval setups where you have positive pairs (e.g. (query, relevant_doc)) as it will sample in each batch n-1 negative docs randomly.

The performance usually increases with increasing batch sizes.

For more detals, see:
* [docs](https://www.sbert.net/docs/package_reference/losses.html)
* [paper]( 

**MultipleNegativesRankingLoss** es una gran función de pérdida si solo tiene pares positivos, por ejemplo, solo pares de textos similares como pares de paráfrasis, pares de preguntas duplicadas, pares de (consulta, respuesta) o pares de (idioma_origen, idioma_destino ).

Esta función de pérdida funciona muy bien para entrenar incrustaciones para configuraciones de recuperación en las que tiene pares positivos (por ejemplo, (consulta, documento_relevante)), ya que tomará muestras en cada lote de n-1 documentos negativos de forma aleatoria.

El rendimiento suele aumentar al aumentar el tamaño de los lotes.

Para obtener más detalles, consulte:
* [docs](https://www.sbert.net/docs/package_reference/losses.html)
* [papel](

In [9]:
from sentence_transformers import losses

In [10]:
loss = losses.MultipleNegativesRankingLoss(model)

### Definir evaluador

Configuramos un evaluador con nuestra división de valores del conjunto de datos para monitorear qué tan bien se está desempeñando el modelo de incorporación durante el entrenamiento.

In [11]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

In [12]:
dataset = val_dataset

corpus = dataset['corpus']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

### Ejecutar entrenamiento

The training loop is very straight forward to steup thanks to sentencetransformers' high-level model training API.
All we need to do is plugging in the data loader, loss function, and evaluator that we defined in the previous cells (along with a couple of additional minor settings).

El ciclo de entrenamiento es muy sencillo de intensificar gracias a la API de entrenamiento de modelos de alto nivel de SentenciaTransformers.
Todo lo que tenemos que hacer es conectar el cargador de datos, la función de pérdida y el evaluador que definimos en las celdas anteriores (junto con un par de configuraciones menores adicionales).

In [13]:
# Entrenamos el modelo durante muy pocas épocas en este ejemplo de prueba.
# Normalmente este valor debería ser mayor para obtener un mejor rendimiento.

EPOCHS = 2

In [14]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='exp_finetune',
    show_progress_bar=True,
    evaluator=evaluator, 
    evaluation_steps=50,
)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/70 [00:00<?, ?it/s]

Iteration:   0%|          | 0/70 [00:00<?, ?it/s]